In [1]:
import os
import sys
sys.path.append(r"/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification")

from tqdm import tqdm
from datetime import datetime
import shutil
import pandas as pd

import brain_mets_classification.config as config
import brain_mets_classification.custom_funcs as funcs

In [2]:
path_to_folder = f"{config.path}/originalPatientFiles"
#path_to_folder = f"{config.path}/RgbBrainMetsSampleN30"
pathToCleanMRIList = ""

In [3]:
#Example file structure
#├── Anonymized - 01005097
#│   └── Mrt Body
#│       ├── Diffusion trace tra schnell_ADC - 8
#│       │   ├── IM-2330-0001-0001.dcm
#│       │   ├── IM-2330-0002-0001.dcm
#...
#│       │   ├── IM-2469-0026-0001.dcm
#│       │   └── IM-2469-0027-0001.dcm
#│       ├── Diffusion trace tra schnell_TRACEW - 7
#│       │   ├── IM-2329-0001-0001.dcm
#│       │   ├── IM-2329-0002-0001.dcm
#...
#│       │   ├── IM-2468-0026-0001.dcm
#│       │   └── IM-2468-0027-0001.dcm
#│       ├── T1 mp-rage3d we sag 1mm KM - 13
#│       │   ├── IM-2335-0001-0001.dcm
#│       │   ├── IM-2335-0002-0001.dcm
#...
#│       │   ├── IM-2474-0159-0001.dcm
#│       │   └── IM-2474-0160-0001.dcm
#├── Anonymized - 12345678
#...

blackList = ["auswertung_fmrt", "fmri", "thorax", "lws", "hws", "bws", "hand", "posdisp", "cor", "sag", "cest", "ciss", "dti", "evidence", "reading", "field", "evaseries", "ct", "lokalizer", "localizer", "mip", "protocol", "resolve", "results", "screen save", "sub", "svs", "tof", "mean_"]

# Creates a new directory for all the patient folders
pathToCleanMRIList = funcs.createNewPreprocessingStepFolder(step=0)

# Goes through list of files/folders at path_to_folder and only adds the directories to the list
folderList = [
    folder for folder in os.listdir(path_to_folder) if os.path.isdir(os.path.join(path_to_folder, folder))
]

patientIDs = []

sequencesList = []


# Loops through all the "Anonymized - #######" folders
for patient_folder in tqdm(folderList):

    # ignores the ds_folders
    if config.dsStore in patient_folder:
        continue

    # all folders are named like "Anonymized - 12345678"
    patientID = patient_folder.split(" - ")[1]

    # adds the patientID to the list patientIDS if it hasn't been added before
    if patientID not in patientIDs:
        patientIDs.append(patientID)

    path_to_MRI_session_folders = os.path.join(path_to_folder, patient_folder)
    MRI_session_folders = os.listdir(path_to_MRI_session_folders)

    # loops through the different MRI sessions
    for mri_session in MRI_session_folders:
        
        # ignores the ds_folders
        if config.dsStore in mri_session:
            continue

        # get a list of all the sequences in the mri_sequences
        path_to_mri_sequences = os.path.join(path_to_MRI_session_folders, mri_session)
        mri_sequences = os.listdir(path_to_mri_sequences)

        # loops through the different sequences created during each MRI session
        for sequence in mri_sequences:

            # ignores the ds_folders
            if config.dsStore in sequence:
                continue

            sequences_lower_cased = sequence.lower()

            if not any(blackListSeq in sequences_lower_cased for blackListSeq in blackList):

                # create folder for patient
                funcs.createFolderForPatient(
                    path = pathToCleanMRIList,
                    patientID = patientID
                )

                # # create new folder as pathToCleanMRIList/patientID/T1CE
                # path_to_sequence = funcs.createSequenceFolder(
                #     path = os.path.join(pathToCleanMRIList, patientID),
                #     patientID = patientID,
                #     sequence = "",
                #     sequence_list = [],
                #     original_sequence_name = sequence
                # )

                pathToPatient = os.path.join(pathToCleanMRIList, patientID)
                folderName = f"{patientID}_{sequence}"
                path_to_sequence = os.path.join(pathToPatient, folderName)
                # check if path_to_sequence already exists then create directory
                sequences = os.listdir(pathToPatient)
                counter = sequences.count(folderName)
                if counter >= 1:
                    path_to_sequence = f"{path_to_sequence}{counter + 1}"
                    os.mkdir(path_to_sequence)
                else:
                    os.mkdir(path_to_sequence)

                # get list of all the dicom files for the T1CE sequence
                dicomFiles = os.listdir(os.path.join(path_to_mri_sequences, sequence))

                # loops through the list of dicom files
                for dicomFile in dicomFiles:
                    # ignores the ds_folders
                    if config.dsStore in dicomFile:
                        continue

                    # copy each file individually into the path_to_sequence folder
                    shutil.copyfile(os.path.join(path_to_mri_sequences, sequence, dicomFile), os.path.join(path_to_sequence, dicomFile))



#             sequencesList.append(sequences_lower_cased)

# cleanList = list(dict.fromkeys(sorted(sequencesList)))

# minusNumberList = []

# for sequence in cleanList:
#     sequenceWithoutNumber = sequence.split(" - ")[0]
#     minusNumberList.append(sequenceWithoutNumber)

# cleanMinusNumber = list(dict.fromkeys(minusNumberList))









# removedSequences = []

# for sequence in cleanMinusNumber:
#     if not any(backListSequ in sequence for backListSequ in blackList): #and any(whiteListSeq in sequence for whiteListSeq in whiteList):
#         print(sequence)
#     else:
#         removedSequences.append(sequence)

# print(f"\n\nRemoved Sequences: {len(removedSequences)}")
# for removedSequence in removedSequences:
#     print(removedSequence)

100%|██████████| 374/374 [29:16<00:00,  4.70s/it] 


removes the patients with less than 4 sequences

In [4]:
filterFolderList = [
    folder for folder in os.listdir(pathToCleanMRIList) if os.path.isdir(os.path.join(pathToCleanMRIList, folder))
]

# loops through all the "12345678" folders
for patient_folder in tqdm(filterFolderList):

    # ignores the ds_folders
    if config.dsStore in patient_folder:
        continue
    
    path_to_patient = os.path.join(pathToCleanMRIList, patient_folder)

    # list to the mri sessions folders
    mri_sessions = os.listdir(path_to_patient)

    if len(mri_sessions) < 4:
        os.system(f"rm -r {path_to_patient}")

100%|██████████| 370/370 [00:04<00:00, 84.25it/s] 


To-Do:
- rename sequences

In [5]:
print(pathToCleanMRIList)

/Volumes/BrainMets/Rgb_Brain_Mets/allPatients/Rgb_Brain_Mets_Preprocessing0_20240109-154442


In [7]:
pathToPreprocess1 = funcs.createNewPreprocessingStepFolder(step=1)

FLAIRList = ["flair"]
KMList = ["km"]
T2SternList = ["stern", "hemo"]
ADCList = ["adc"]
DWIList = ["diffusion"]

# list of patientIDs
folderPreprocess0List = [
    folder for folder in os.listdir(pathToCleanMRIList) if os.path.isdir(os.path.join(pathToCleanMRIList, folder))
]

# Loops through all the "Anonymized - #######" folders
for patient_folder in tqdm(folderPreprocess0List):

    # ignores the ds_folders
    if config.dsStore in patient_folder:
        continue
    
    path_to_patient_sequences = os.path.join(pathToCleanMRIList, patient_folder)
    mri_sequences = os.listdir(path_to_patient_sequences)

    # create new folder for patient in pathToPreprocess1

    # loop through the sequences for each patient
    for sequence in mri_sequences:

        # only get the sequence name, folder names: "12345678_SEQUENCENAME"
        sequence_lower_cased = sequence.split("_")[1].lower()

        # check if the name fits to a sequence
        # if so: create a new folder name in pathToPreprocess1 / patientID with the updated sequence name
        # find a way to note how many sequences of each kind exist for each patient
        # create a list of patients with sequences that didn't fit the criteria
        # look through those patients manually
        # ?
        # profit

    

100%|██████████| 321/321 [00:00<00:00, 6870.72it/s]

diffusion trace tra schnell
t2 tse tra schnell - 6
t1 se tra schnell - 10
t2 stern flash tra schnell - 9
t2 flair tra schnell - 5
t1 se tra km schnell - 11
diffusion trace tra schnell
diffusion trace tra schnell
t2 tse tra schnell - 6
t1 se tra schnell - 10
t2 stern flash tra schnell - 9
t2 flair tra schnell - 5
t1 se tra km schnell - 11
diffusion trace tra schnell
t1 se tra schnell - 6
diffusion trace tra schnell - 4
t2 flair tra schnell - 2
diffusion trace tra schnell
t2 tse tra schnell - 3
t1 se tra km schnell - 7
diffusion trace tra schnell
t2 tse tra schnell - 6
t1 se tra schnell - 10
t2 stern flash tra schnell - 9
t2 flair tra schnell - 5
t1 se tra km schnell - 11
diffusion trace tra schnell
t2
t1
ep2d
t1
ep2d
diffusion trace tra schnell
t2 tse tra schnell - 6
t1 se tra schnell - 10
t2 stern flash tra schnell - 9
t2 flair tra schnell - 5
t1 se tra km schnell - 11
diffusion trace tra schnell
diffusion trace tra schnell
t2 tse tra schnell - 6
t1 se tra schnell - 10
t2 stern flash t